# Starting a binary black hole simulation

One of the systems that we are simulating with SpECTRE are binary black hole
(BBH) mergers and their gravitational waves at very high accuracy. In this
tutorial we will start a BBH simulation with SpECTRE and monitor its output.

First, make sure that you have compiled all the executables you need for running
BBH simulations:

```sh
make -j24 cli bbh
```

We will use the SpECTRE command-line interface (CLI) in this session. You have
learned how to access the CLI in the technology check session. Try it now by
exploring the `spectre bbh` subcommand of the CLI:

In [1]:
!spectre bbh --help

Usage: spectre bbh [OPTIONS] COMMAND [ARGS]...

  Pipeline for binary black hole simulations.

Options:
  -h, --help  Show this message and exit.

Commands:
  find-horizon    Find an apparent horizon in volume data.
  generate-id     Generate initial data for a BBH simulation.
  postprocess-id  Postprocess initial data after generation.
  start-inspiral  Schedule an inspiral simulation from initial data.
  start-ringdown  Schedule a ringdown simulation from the inspiral.


These subcommands are part of the BBH pipeline, which you can use to start and
control BBH simulations. Steps in the BBH pipeline are, for example, generating
the initial data, running the inspiral simulation, running the ringdown
simulation, and extracting waveforms.

> Note: all CLI commands are also available in Python, so everything you do with
> the CLI you can also do in Python scripts. We will use Python to work with BBH
> data in the next session.

## Generate initial data

Using the pipeline, you can generate initial data to start your BBH simulation
like this:

```sh
spectre bbh generate-id -q 1 --chi-A 0 0 0 --chi-B 0 0 0 -D 16 -w 0.015 -a 0 \
  --no-control -d ./01_bbh
```

That's it! Now your job is scheduled on the cluster. Once the job runs, it will
generate initial data with the parameter you have specified. To get initial data
quickly we have disabled the control loop that drives the horizon masses and
spins to your specified parameters with `--no-control`. We could also add
`--evolve` to start the evolution of the initial data right after generation.
However, we'll start the inspiral in a separate command below to control the
parameters.

The initial data will be generated in the _pipeline directory_ (`-d`) that we
set to `./01_bbh` in the command above. You will find a directory named
`001_InitialData` in the pipeline directory. The next step in the pipeline will
create `002_Inspiral`. If you're running just a single step of the pipeline, you
can also specify the output directory directly with `-o`.

Of course you can change parameters such as the mass-ratio $q$, the black hole
spins $\mathbf{\chi}_A$ and $\mathbf{\chi}_B$, and the initial orbital
parameters:

- Coordinate separation $D_0$ (`-D`)
- Orbital angular velocity $\Omega_0$ (`-w`)
- Radial expansion velocity $\dot{a}_0$ (`-a`)

Alternatively, you can choose `--eccentricity=0` plus either `--num-orbits`, or
`--time-to-merger`, or one of the above orbital parameters to define the orbit
based on post-Newtonian estimates.

> To learn more about initial data generation you will have to wait until
> Friday.

The initial data generation should take about 4.5 minutes to complete at this
(fairly high) resolution on the 32 cores that we have available on Oscar.

## Starting the inspiral

The steps of the pipeline are connected, e.g. you can pass `--evolve` to
`spectre bbh generate-id` to automatically start the inspiral right after
initial data generation. However, you can also run each step in the pipeline on
its own, e.g. to start the inspiral with different parameters or in another
directory. Let's start the inspiral on just 1 node (with 32 cores), because
that's what we have available on Oscar:

```sh
spectre bbh start-inspiral 01_bbh/001_InitialData/InitialData.yaml \
  -N 1 -d ./01_bbh
```

This will create `002_Inspiral` in your pipeline directory. In it, the first
_segment_ of your BBH simulation will start. A segment is a chunk of a
long-running simulation, which will terminate after some time (typically 1 day)
and resubmit itself so we can run on supercomputers with time-limited queues.

Note that you can also start the inspiral from SpEC data (if you happen to have
some):

```sh
spectre bbh start-inspiral path/to/ID_Params.perl -O bbh_spec
```

## Monitoring your simulation

Now you can monitor your simulation. Try running:

```sh
spectre status
```

It will show you the state of your running simulations on this machine.

> **Pro tip:** Try `spectre status -w 5` in a separate terminal to keep watch of
> your simulations, updating every 5 seconds.

You can also run a dashboard to give you a visual summary of your running
simulations with lots of plots (this is still very experimental):

```sh
python-spectre -m streamlit run $SPECTRE_HOME/tools/Status/Dashboard.py
```

We have the dashboard running on Oscar, so if you're feeling adventorous you can
forward port 8501 on Oscar's `vscode1` node and open the dashboard in your
browser:

- Dashboard URL: http://localhost:8501

### Example: Plot trajectories

You can use the spectre CLI to inspect the output data of the simulation while
it is running. We will explore simulation data in detail in the next tutorial.
For now, just try to plot the trajectories:

```sh
spectre plot trajectories 01_bbh/002_Inspiral/Segment_*/BbhReductions.h5 \
  -o trajectories.pdf
```

> Note: If you want to try this before your simulation has sufficiently
> progressed to show anything interesting, you can look at the complete
> simulation data in this directory:
>
> ```sh
> export BBH_DATA=/oscar/data/icerm/knelli/workshop_materials/01_monday/bbh_inspiral
> spectre plot trajectories $BBH_DATA/BbhReductions.h5 -o trajectories.pdf
> ```

![Trajectories](images/trajectories.png)

> **Pro tip:** You can also regularly regenerate plots like this to monitor your
> simulation. Try `watch spectre plot trajectories ...`.

## Inspecting the input file

SpECTRE simulations are defined by an input file in YAML format. The input file
collects all options for the simulation and also specifies the C++ executable to
run it.

The inspiral input file that you just started running is here:

```
01_bbh/002_Inspiral/Segment_0000/Inspiral.yaml
```

Open the input file to inspect it in an editor of your choice (or just print it
with `cat path/to/input/file`).



## Reproducibility of simulations

A quick note on reproducibility, which is a crucial topic in scientific
computing:

SpECTRE simulations are reasonably well reproducible because (1) all options are
defined in the YAML input file, and (2) the input file and even the entire
source code used to compile the executable are stored in each H5 output file.

So, for the highest level of reproducibility you can extract the input file from
an H5 output file (see `spectre extract-input --help`), and also extract the
source code archive (see
[docs](https://spectre-code.org/tutorial_visualization.html#autotoc_md93)), then
compile and run the executable.

To help with reproducibility of results you publish in papers, we recommend that
you publish your input files as supplementary material and also cite the
specific code release with which you ran your simulations (see [Versioning and
releases](https://spectre-code.org/versioning_and_releases.html)).

## Troubleshooting a BBH simulation

If something fails (and of course it will), you can start by looking through the
data you have. First, look at the log file of your simulation. It's often named
`spectre.out` and located in the run directory of your simulation. Go to the
very end of the file and look for error messages. If you see many error
messages, the first one is often most helpful. Ideally, you'll find a stack
trace with the function calls that lead to the error. The error message and the
stack trace might already give you an idea as to what has happened.

It will also be helpful to analyse the output data that your simulation has
written up to this point, and look for anything fishy. In the next tutorial we
will go through a lot of ways to analyse output data from your simulations.

Once you have collected some clues as to what as happened, the best place to ask
for help is the `#spectre` channel on the SXS Slack. See you there!

## Eccentricity reduction

We have basic support for measuring the orbital eccentricity and updating the
initial orbital parameters to reduce the eccentricity. Work on automation and
improvements of this procedure are ongoing, and mostly independent of the actual
simulation code. You can do a quick estimate of the orbital eccentricity with
this:

```sh
spectre eccentricity-control 01_bbh/002_Inspiral/Segment_0000/BbhReductions.h5 \
  --tmin 500 --tmax 2000 -o eccentricity.pdf
```